In [3]:
from transformers import AutoTokenizer, BertForQuestionAnswering, BertModel
import torch
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# Load model and retrieve attention weights

from bertviz import head_view, model_view
# from transformers import BertTokenizer, BertModel
from transformers import AutoTokenizer, BertForQuestionAnswering, AutoModel

model_version = 'sentence-transformers/all-mpnet-base-v2'
model = AutoModel.from_pretrained(model_version, output_attentions=True, output_hidden_states=True)
tokenizer = AutoTokenizer.from_pretrained(model_version)
question = "he is a good man"
text = "he is a good man"
sentences=["he is a good man", "he is a good man"]
inputs = tokenizer(sentences, return_tensors='pt')
input_ids = inputs['input_ids']
print(inputs.keys())
attention = model(input_ids)[-1]
# sentence_b_start = token_type_ids[0].tolist().index(1)
input_id_list = input_ids[0].tolist() # Batch index 0
tokens = tokenizer.convert_ids_to_tokens(input_id_list) 

dict_keys(['input_ids', 'attention_mask'])


In [26]:
inputs

{'input_ids': tensor([[   0, 2006, 2007, 1041, 2208, 2162,    2],
        [   0, 2006, 2007, 1041, 2208, 2162,    2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1]])}

In [4]:
ticks = []
for j in range(len(inputs.input_ids)):
    print(f'Sentence {j+1}')
    for i in range(len(inputs.input_ids[j])):
        print(inputs.input_ids[0][i], ':', tokenizer.decode(inputs.input_ids[0][i]))
        ticks.append(tokenizer.decode(inputs.input_ids[0][i]))

Sentence 1
tensor(0) : <s>
tensor(2006) : he
tensor(2007) : is
tensor(1041) : a
tensor(2208) : good
tensor(2162) : man
tensor(2) : </s>
Sentence 2
tensor(0) : <s>
tensor(2006) : he
tensor(2007) : is
tensor(1041) : a
tensor(2208) : good
tensor(2162) : man
tensor(2) : </s>


In [17]:
import torch.nn.functional as F

#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


# Sentences we want sentence embeddings for
sentences = ['smart', 'rock']

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-mpnet-base-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-mpnet-base-v2')

# Tokenize sentences
encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

# Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)

# Perform pooling
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

# Normalize embeddings
sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

print('Sentence Embeddings:', sentence_embeddings)
print('Cosine Distance from embeddings generated by the model: {:4f}'.format(torch.cosine_similarity(sentence_embeddings[0].unsqueeze(0), sentence_embeddings[1].unsqueeze(0)).item()))

Sentence Embeddings: tensor([[-0.0164, -0.0189, -0.0140,  ...,  0.0128, -0.0199, -0.0272],
        [-0.0277,  0.0504, -0.0090,  ...,  0.0323, -0.0414, -0.0181]])
Cosine Distance from embeddings generated by the model: 0.147754


In [18]:
attention_tensors = []

for tensor in model(input_ids).attentions:
    attention_tensors.append(tensor.detach().numpy())

attentions_temp = torch.tensor(np.array(attention_tensors))
print(attentions_temp.shape)

torch.Size([12, 2, 12, 7, 7])


In [20]:
sentence1_len = len(question.split(" "))


In [21]:
sns.heatmap(attentions_temp.mean(axis=2)[-2][0].T[1:sentence1_len+2,sentence1_len+2:] ,xticklabels = ticks[sentence1_len+2:] , yticklabels = ticks[1:sentence1_len+2])

ValueError: zero-size array to reduction operation fmin which has no identity

In [ ]:
attentions_temp.mean(axis=2)[-2][0].T[1:sentence1_len+2,sentence1_len+2:]

tensor([[0.0421, 0.0409, 0.0336, 0.0095, 0.0172, 0.0474],
        [0.0193, 0.0492, 0.0426, 0.0117, 0.0106, 0.0570],
        [0.0120, 0.0302, 0.0290, 0.0100, 0.0077, 0.0190],
        [0.0188, 0.0537, 0.0471, 0.0344, 0.0193, 0.0451],
        [0.0439, 0.0358, 0.0449, 0.0202, 0.0317, 0.0672],
        [0.0240, 0.0332, 0.0316, 0.0130, 0.0147, 0.0244]])

In [ ]:
tokenizer(question, text, return_tensors='pt')

{'input_ids': tensor([[ 101, 2002, 2003, 1037, 2204, 2158,  102, 2002, 2003, 1037, 2204, 2158,
          102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [4]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

In [5]:

import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel
 
 
def get_word_idx(sent: str, word: str):
    return sent.split(" ").index(word)
 
 
def get_hidden_states(encoded, token_ids_word, model, layers):
    """Push input IDs through model. Stack and sum `layers` (last four by default).
        Select only those subword token outputs that belong to our word of interest
        and average them."""
    with torch.no_grad():
        output = model(**encoded)
 
     # Get all hidden states
    states = output.hidden_states
     # Stack and sum all requested layers
    output = torch.stack([states[i] for i in layers]).sum(0).squeeze()
     # Only select the tokens that constitute the requested word
    word_tokens_output = output[token_ids_word]
 
    return word_tokens_output.mean(dim=0)
 
 
def get_word_vector(sent, idx, tokenizer, model, layers):
    """Get a word vector by first tokenizing the input sentence, getting all token idxs
        that make up the word of interest, and then `get_hidden_states`."""
    encoded = tokenizer.encode_plus(sent, return_tensors="pt")
     # get all token idxs that belong to the word of interest
    token_ids_word = np.where(np.array(encoded.word_ids()) == idx)
 
    return get_hidden_states(encoded, token_ids_word, model, layers)
 
 
def main(layers , sent1 , sent2):
    # Use last four layers by default
    layers = [-4, -3, -2, -1] if layers is None else layers
    tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
    model = AutoModel.from_pretrained("bert-base-cased", output_hidden_states=True)

    sent1_embeddings = []
    sent2_embeddings = []
    sent1 = sent1
    sent2 = sent2
    for w in sent1.split(" "):
        sent1_embeddings.append(get_word_vector(sent1, get_word_idx(sent1, w), tokenizer, model, layers))
    for w in sent2.split(" "):
        sent2_embeddings.append(get_word_vector(sent2 , get_word_idx(sent2, w), tokenizer, model, layers))
    # idx = get_word_idx(sent, "cookies")

    # word_embedding = get_word_vector(sent, idx, tokenizer, model, layers)
     
    return sent1_embeddings, sent2_embeddings 


In [8]:
sent1 = question
sent2 = text
s1 , s2 = main(None , sent1 , sent2)
sentence1 = sent1.split(" ")
sentence2 = sent2.split(" ")
for idx1 , v1 in enumerate(s1):
    for idx2 , v2 in enumerate(s2):
        print('{} ~ {} ; Cosine Distance: {:5f}' .format(sentence1[idx1], sentence2[idx2], torch.cosine_similarity(v1.unsqueeze(0), v2.unsqueeze(0)).item()))

he ~ he ; Cosine Distance: 1.000000
he ~ is ; Cosine Distance: 0.790254
he ~ a ; Cosine Distance: 0.717984
he ~ good ; Cosine Distance: 0.688151
he ~ man ; Cosine Distance: 0.763715
is ~ he ; Cosine Distance: 0.790254
is ~ is ; Cosine Distance: 1.000000
is ~ a ; Cosine Distance: 0.865570
is ~ good ; Cosine Distance: 0.786840
is ~ man ; Cosine Distance: 0.787070
a ~ he ; Cosine Distance: 0.717984
a ~ is ; Cosine Distance: 0.865570
a ~ a ; Cosine Distance: 1.000000
a ~ good ; Cosine Distance: 0.800178
a ~ man ; Cosine Distance: 0.746675
good ~ he ; Cosine Distance: 0.688151
good ~ is ; Cosine Distance: 0.786840
good ~ a ; Cosine Distance: 0.800178
good ~ good ; Cosine Distance: 1.000000
good ~ man ; Cosine Distance: 0.762201
man ~ he ; Cosine Distance: 0.763715
man ~ is ; Cosine Distance: 0.787070
man ~ a ; Cosine Distance: 0.746675
man ~ good ; Cosine Distance: 0.762201
man ~ man ; Cosine Distance: 1.000000


In [ ]:
attention_second_last = attentions_temp.mean(axis=2)[-2][0].T[1:sentence1_len+2,sentence1_len+2:] 
sum = 0
for r in range(attention_second_last.shape[0]-1):
    max_idx = np.argmax(attention_second_last[r])
    print(r , sentence1[r] , max_idx)
    if max_idx == len(attention_second_last[r])-1:
        continue
    print(sentence1[r] , sentence2[max_idx] , torch.cosine_similarity(s1[r].unsqueeze(0), s2[max_idx].unsqueeze(0)))
    sum+=torch.cosine_similarity(s1[r].unsqueeze(0), s2[max_idx].unsqueeze(0))


print(sum)
print(sum/len(question.split(" ")))

0 he tensor(5)
1 is tensor(5)
2 a tensor(1)
a is tensor([0.8656])
3 good tensor(1)
good is tensor([0.7868])
4 man tensor(5)
tensor([1.6524])
tensor([0.3305])
